In [ ]:
%pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahmedwaelnasef/cars-dataset")
!mkdir data
!cp {path}/cars.csv data/cars.csv


In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
df = pd.read_csv('data/cars.csv')
df.head()
df['seats'].value_counts()  


seats
5 Seater     3471
4 Seater      847
7 Seater      532
2 Seater      428
8 Seater      211
             ... 
14.0            1
13 Seater       1
230             1
220             1
2.8             1
Name: count, Length: 82, dtype: int64

In [3]:
string_columns = df.select_dtypes(include=['object']).columns
df[string_columns] = df[string_columns].apply(lambda x: x.str.strip())

In [ ]:
df.shape[0] - df[df['price'].str.contains(r'\d', regex=True)].shape[0]

In [ ]:
dataframe_no_price = df[df['price'].str.contains(r'^\D*$', regex=True) == True]['price']
dataframe_no_price.unique()

In [ ]:
dataframe_no_price.nunique()

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6292 entries, 0 to 6307
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   car name         6292 non-null   object
 1   price            6292 non-null   object
 2   engine_capacity  6292 non-null   object
 3   cylinder         5668 non-null   object
 4   horse_power      6292 non-null   object
 5   top_speed        6292 non-null   object
 6   seats            6292 non-null   object
 7   brand            6292 non-null   object
 8   country          6292 non-null   object
dtypes: object(9)
memory usage: 491.6+ KB


In [ ]:
df['price'] = df['price'].astype(float)

In [13]:
df.isna().sum()

car name             0
engine_capacity      0
cylinder           624
horse_power          0
top_speed            0
seats                0
brand                0
country              0
currency             0
price                2
dtype: int64

In [6]:
# Extract currency and number
pattern = r'([A-Z]+)\s([\d,]+)'
extracted = df['price'].str.extract(pattern, expand=True)

# Rename columns for clarity
extracted.columns = ['currency', 'amount']

df = pd.concat([df, extracted], axis=1)
df.drop(columns='price', inplace=True)

df.rename(columns={'amount': 'price'}, inplace=True)

# # Handle non-matching rows (e.g., fill NaN with 'Unknown' for currency and 0 for amount)
# extracted['currency'].fillna('Unknown', inplace=True)
# extracted['amount'].fillna('0', inplace=True)

# print(extracted)
df.head()

,car name,engine_capacity,cylinder,horse_power,top_speed,seats,brand,country,currency,price
0,Fiat 500e 2021 La Prima,0.0,"N/A, Electric",Single,Automatic,150,fiat,ksa,NaN,NaN
1,Peugeot Traveller 2021 L3 VIP,2.0,4,180,8 Seater,8.8,peugeot,ksa,SAR,"140,575"
2,Suzuki Jimny 2021 1.5L Automatic,1.5,4,102,145,4 Seater,suzuki,ksa,SAR,"98,785"
3,Ford Bronco 2021 2.3T Big Bend,2.3,4,420,4 Seater,7.5,ford,ksa,SAR,"198,000"
4,Honda HR-V 2021 1.8 i-VTEC LX,1.8,4,140,190,5 Seater,honda,ksa,NaN,NaN


In [9]:
df['price'] = df['price'].str.replace(',', '').astype(float)

In [10]:


# Dictionary for mapping country to currency
country_to_currency = {
    'ksa': 'SAR',      # Saudi Riyal
    'egypt': 'EGP',    # Egyptian Pound
    'bahrain': 'BHD',  # Bahraini Dinar
    'qatar': 'QAR',    # Qatari Riyal
    'oman': 'OMR',     # Omani Rial
    'kuwait': 'KWD',   # Kuwaiti Dinar
    'uae': 'AED'       # United Arab Emirates Dirham
}

# Dictionary for mapping currency to USD exchange rate
currency_to_usd = {
    'SAR': 0.27,  # Saudi Riyal to USD
    'EGP': 0.032, # Egyptian Pound to USD
    'BHD': 2.65,  # Bahraini Dinar to USD
    'QAR': 0.27,  # Qatari Riyal to USD
    'OMR': 2.60,  # Omani Rial to USD
    'KWD': 3.30,  # Kuwaiti Dinar to USD
    'AED': 0.27   # United Arab Emirates Dirham to USD
}

# Map country to currency
df['currency'] = df['country'].map(country_to_currency)

df['price'] = df.apply(lambda row: row['price'] * currency_to_usd[row['currency']], axis=1)

# Compute the mean price for each country
mean_prices = df.groupby('car name')['price'].transform('mean')

# Replace only the missing values with the mean price of their respective groups
df['price'] = df['price'].fillna(mean_prices)


# Convert prices to USD
df.head()

,car name,engine_capacity,cylinder,horse_power,top_speed,seats,brand,country,currency,price
0,Fiat 500e 2021 La Prima,0.0,"N/A, Electric",Single,Automatic,150,fiat,ksa,SAR,NaN
1,Peugeot Traveller 2021 L3 VIP,2.0,4,180,8 Seater,8.8,peugeot,ksa,SAR,37955.25
2,Suzuki Jimny 2021 1.5L Automatic,1.5,4,102,145,4 Seater,suzuki,ksa,SAR,26671.95
3,Ford Bronco 2021 2.3T Big Bend,2.3,4,420,4 Seater,7.5,ford,ksa,SAR,53460.00
4,Honda HR-V 2021 1.8 i-VTEC LX,1.8,4,140,190,5 Seater,honda,ksa,SAR,NaN


In [12]:
median_na = df.groupby('brand')['price'].transform('median')
df['price'] = df['price'].fillna(median_na)


In [14]:
df.head()

,car name,engine_capacity,cylinder,horse_power,top_speed,seats,brand,country,currency,price
0,Fiat 500e 2021 La Prima,0.0,"N/A, Electric",Single,Automatic,150,fiat,ksa,SAR,19610.000
1,Peugeot Traveller 2021 L3 VIP,2.0,4,180,8 Seater,8.8,peugeot,ksa,SAR,37955.250
2,Suzuki Jimny 2021 1.5L Automatic,1.5,4,102,145,4 Seater,suzuki,ksa,SAR,26671.950
3,Ford Bronco 2021 2.3T Big Bend,2.3,4,420,4 Seater,7.5,ford,ksa,SAR,53460.000
4,Honda HR-V 2021 1.8 i-VTEC LX,1.8,4,140,190,5 Seater,honda,ksa,SAR,28179.975


In [15]:
df.dropna(subset=['price'], inplace=True)
df.isna().sum()

car name             0
engine_capacity      0
cylinder           624
horse_power          0
top_speed            0
seats                0
brand                0
country              0
currency             0
price                0
dtype: int64

In [ ]:
df.drop(columns=['currency', 'price'], inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
indices = df[df['top_speed'].str.contains('Seater')].index

temp = df.loc[indices, 'seats']
df.loc[indices, 'seats'] = df['top_speed']
df.loc[indices, 'top_speed'] = temp

In [ ]:
df['top_speed'].str.contains('Seater').sum()

In [ ]:
df.loc[indices, 'top_speed'].head(20)

In [ ]:
df.isna().sum()

In [ ]:
df[df['price_usd'].isna()]

In [ ]:
df[df['brand'] == 'dorcen']

In [ ]:
df['seats'].value_counts()

In [ ]:
df['car name'].value_counts()

In [ ]:
df[df['car name'] == 'BMW 5 Series 2021 520i']

In [ ]:
# Columns to process
columns = ['seats', 'engine_capacity', 'brand', 'top_speed', 'cylinder', 'horse_power']

# Function to get the mode (most frequent value)
def get_mode(series):
    mode = series.mode()
    return mode[0] if not mode.empty else None

df[columns] = df.groupby('car name')[columns].transform(get_mode)

In [ ]:
df[df['car name'] == 'BMW 5 Series 2021 520i']

In [ ]:
df['price']

In [ ]:
df[df['car name'].duplicated()]

In [ ]:
df['country'].unique()

In [ ]:
df[df['car name'] == 'Aston Martin DBS Superleggera 2021 5.2T V12']

In [ ]:
df['car name'].nunique()

In [ ]:
df['country'].nunique()

In [ ]:
df.loc[indices]

In [ ]:
df[df['car name'] == 'Fiat 500e 2021 La Prima']

In [ ]:
df[df['car name'] == 'Peugeot Traveller 2021 L3 VIP']

In [ ]:
df['seats'].head()

In [ ]:
df['seats'] = df['seats'].apply(lambda x: re.findall(r'\d+', x)[0] if type(x) == str else x)

In [ ]:
df.info()


In [ ]:
df.head(5)

In [ ]:
# mask = df['top_speed'].str.contains('Seater', case=False)
# df.drop(df[mask].index, inplace=True)
# df.reset_index(drop=True, inplace=True)

In [ ]:
df['seats'] = df['seats'].apply(lambda x: x if 'seater' not in x.lower() else x[0])

In [ ]:
# Check if 'seats' column contains no numbers
mask = df['seats'].str.contains(r'^\D+$', regex=True)

mask.sum()
# df[mask==True]

In [ ]:
df['seats'].replace('N A', np.nan, inplace=True)

In [ ]:
df[mask]

In [ ]:
df.drop(df[mask==True].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df


In [ ]:
df['seats'].isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
index = df['cylinder'].str.contains('Electric', case=False).index
df.loc[index, 'cylinder'] = 0

In [ ]:
df.head(5)

In [ ]:
df['cylinder'].value_counts()

In [ ]:
df['country'].value_counts()

In [ ]:
df['brand'].value_counts()

In [ ]:
# df['cylinder'] = df['cylinder'].replace('N/A, Electric', pd.NA)
# df.isna().sum()

In [ ]:
df.dropna(inplace=True)


In [ ]:
df['price'].value_counts()

In [ ]:
df['top_speed'].value_counts()
df['seats'].value_counts()

In [ ]:
 

df['seats'] = df['seats'].str.extract('(\d+)')

df['seats'].value_counts()